In [1]:
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt


def data_load():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    return X_train, y_train, X_test, y_test

def data_preview(X_train):
    fig = plt.figure(figsize=(20,5))
    for i in range(36):
        ax=fig.add_subplot(3, 12, i+1, xticks=[], yticks=[])
        ax.imshow(np.squeeze(X_train[i]))

def normalize(X_train, X_test):
    X_train = X_train.astype('float32')/255
    X_test = X_test.astype('float32')/255
    return X_train, X_test

def one_hot(y_train, y_test):
    num_classes=len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return y_train, y_test

def split(X_train, y_train, valid_nums=5000):
    X_train, X_valid = X_train[valid_nums:], X_train[:valid_nums]
    y_train, y_valid = y_train[valid_nums:], y_train[:valid_nums]
    return X_train, X_valid, y_train, y_valid
    
def mini_AlexNet():
    cnn = Sequential()
    cnn.add(Conv2D(filters=16, kernel_size=2, padding='same',\
        activation='relu', input_shape = (32, 32, 3)))
    cnn.add(MaxPooling2D(pool_size=2))
    cnn.add(Conv2D(filters=32, kernel_size=2, padding='same',\
        activation='relu', input_shape = (32, 32, 3)))
    cnn.add(MaxPooling2D(pool_size=2))
    cnn.add(Conv2D(filters=64, kernel_size=2, padding='same',\
        activation='relu', input_shape = (32, 32, 3)))
    cnn.add(MaxPooling2D(pool_size=2))
    
    cnn.add(Dropout(rate=0.3))

    cnn.add(Flatten())

    cnn.add(Dense(500, activation='relu'))
    cnn.add(Dropout(rate=0.4))

    cnn.add(Dense(10, activation='softmax'))
    return cnn

In [2]:
X_train, y_train, X_test, y_test=data_load()

X_train, X_test = normalize(X_train, X_test)

y_train, y_test = one_hot(y_train, y_test)

X_train, X_valid, y_train, y_valid = split(X_train, y_train)

model=mini_AlexNet()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',\
    metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)

hist= model.fit(X_train, y_train, batch_size=32, epochs=100,\
    validation_data=(X_valid, y_valid), callbacks=[checkpointer],\
        verbose=2, shuffle=True)

Epoch 1/100

Epoch 1: val_loss improved from inf to 1.42570, saving model to model.weights.best.hdf5
1407/1407 - 19s - loss: 1.6390 - accuracy: 0.4008 - val_loss: 1.4257 - val_accuracy: 0.4784 - 19s/epoch - 13ms/step
Epoch 2/100

Epoch 2: val_loss improved from 1.42570 to 1.39785, saving model to model.weights.best.hdf5
1407/1407 - 19s - loss: 1.3213 - accuracy: 0.5282 - val_loss: 1.3978 - val_accuracy: 0.5134 - 19s/epoch - 13ms/step
Epoch 3/100

Epoch 3: val_loss improved from 1.39785 to 1.14783, saving model to model.weights.best.hdf5
1407/1407 - 17s - loss: 1.1862 - accuracy: 0.5786 - val_loss: 1.1478 - val_accuracy: 0.5912 - 17s/epoch - 12ms/step
Epoch 4/100

Epoch 4: val_loss improved from 1.14783 to 1.06704, saving model to model.weights.best.hdf5
1407/1407 - 17s - loss: 1.1082 - accuracy: 0.6093 - val_loss: 1.0670 - val_accuracy: 0.6276 - 17s/epoch - 12ms/step
Epoch 5/100

Epoch 5: val_loss improved from 1.06704 to 1.02083, saving model to model.weights.best.hdf5
1407/1407 - 15s

In [4]:
model.load_weights('model.weights.best.hdf5')

score = model.evaluate(X_test, y_test, verbose=0)
print('\n', "Test Accuracy:", score[1])


 Test Accuracy: 0.7070000171661377
